# Go Benchmark Results Analyzer

This notebook analyzes Go benchmark output files, extracting performance metrics and generating visualizations.

## Features
- **Automatic folder selection**: Automatically analyzes the latest benchmark run if no folder is specified
- Parse Go benchmark output files
- Extract performance metrics (ns/op, MB/s, B/op, allocs/op)
- Compare SIMD vs Fallback performance
- Visualize throughput, memory usage, and scaling behavior
- Support for custom metrics in benchmark output
- CPU profile analysis (.prof files)
- Export results to CSV for further analysis

## Usage
By default, this notebook automatically selects the most recent `run_*_analysis` folder based on timestamp. To analyze a specific folder, set the `RUN_FOLDER` parameter in the configuration cell below.

In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from pathlib import Path
from datetime import datetime

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

## Available Benchmark Files

Find all benchmark result files in the project.

In [ ]:
# Configuration: Select files for this analysis session
# Group related files together for a single investigation

# ============================================================================
# BENCHMARK FILE SELECTION
# ============================================================================
# PARAMETER: Set RUN_FOLDER to specify which analysis folder to use
# If set to None, the latest timestamped folder will be automatically selected

RUN_FOLDER = None  # Set to None to auto-select latest, or specify path like "results/run_20251031_111520_analysis"

# ============================================================================
# AUTO-SELECT LATEST FOLDER IF NOT SPECIFIED
# ============================================================================
def find_latest_analysis_folder(results_dir="results"):
    """
    Find the most recent analysis folder based on timestamp in folder name.

    Returns:
        Path to the latest folder, or None if no folders found
    """
    import glob
    import re

    # Find all run_* folders
    pattern = os.path.join(results_dir, "run_*_analysis")
    folders = glob.glob(pattern)

    if not folders:
        return None

    # Extract timestamps and sort
    folder_times = []
    for folder in folders:
        basename = os.path.basename(folder)
        # Extract timestamp: run_YYYYMMDD_HHMMSS_analysis
        match = re.search(r'run_(\d{8}_\d{6})_analysis', basename)
        if match:
            timestamp = match.group(1)
            folder_times.append((timestamp, folder))

    if not folder_times:
        return None

    # Sort by timestamp (descending) and return the latest
    folder_times.sort(reverse=True)
    return folder_times[0][1]

if RUN_FOLDER is None:
    RUN_FOLDER = find_latest_analysis_folder()
    if RUN_FOLDER:
        print("=" * 80)
        print("AUTO-SELECTED LATEST ANALYSIS FOLDER")
        print("=" * 80)
        print(f"Using: {RUN_FOLDER}")
        print("\nTo analyze a different folder, set RUN_FOLDER in this cell to:")
        print('  RUN_FOLDER = r"results\\run_YYYYMMDD_HHMMSS_analysis"')
        print("=" * 80)
    else:
        print("=" * 80)
        print("ERROR: No analysis folders found")
        print("=" * 80)
        print("No 'run_*_analysis' folders found in results/")
        print("Please run the benchmark script first:")
        print("  bash scripts/benchmark.sh")
        print("=" * 80)
        raise FileNotFoundError("No analysis folders found in results/")

# Set up file paths based on selected folder
BENCHMARK_FILE = os.path.join(RUN_FOLDER, "benchmark_full_suite.txt")
PROFILE_FILE = os.path.join(RUN_FOLDER, "cpu_profile.prof")
PROFILE_TREE_FILE = os.path.join(RUN_FOLDER, "profile_tree.txt")

# ============================================================================
# ANALYSIS CONFIGURATION
# ============================================================================
# Extract analysis name from folder
ANALYSIS_NAME = os.path.basename(RUN_FOLDER).replace('_analysis', '')

# ============================================================================
# VALIDATION
# ============================================================================
print("\n" + "=" * 80)
print("ANALYSIS CONFIGURATION")
print("=" * 80)
print(f"\nAnalysis Folder: {RUN_FOLDER}")
print(f"Analysis Name: {ANALYSIS_NAME}")

if not os.path.exists(BENCHMARK_FILE):
    print(f"\n✗ ERROR: Benchmark file not found: {BENCHMARK_FILE}")
    print("Please run the benchmark script first:")
    print("  bash scripts/benchmark.sh")
    raise FileNotFoundError(f"Benchmark file not found: {BENCHMARK_FILE}")
else:
    print(f"\n✓ Benchmark file: {os.path.basename(BENCHMARK_FILE)}")
    print(f"  Size: {os.path.getsize(BENCHMARK_FILE):,} bytes")

if PROFILE_FILE:
    if not os.path.exists(PROFILE_FILE):
        print(f"\n○ CPU Profile: Not found (analysis will be skipped)")
        PROFILE_FILE = None
    else:
        print(f"\n✓ CPU Profile: {os.path.basename(PROFILE_FILE)}")
        print(f"  Size: {os.path.getsize(PROFILE_FILE):,} bytes")
else:
    print(f"\n○ CPU Profile: None")

if PROFILE_TREE_FILE:
    if not os.path.exists(PROFILE_TREE_FILE):
        print(f"\n○ Profile Tree: Not found (analysis will be skipped)")
        PROFILE_TREE_FILE = None
    else:
        print(f"\n✓ Profile Tree: {os.path.basename(PROFILE_TREE_FILE)}")
        print(f"  Size: {os.path.getsize(PROFILE_TREE_FILE):,} bytes")
else:
    print(f"\n○ Profile Tree: None")

# Set up output directory - output goes inside the run folder
output_dir = RUN_FOLDER
print(f"\n✓ Output directory: {output_dir}")

os.makedirs(os.path.join(output_dir, 'graphs'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'data'), exist_ok=True)

print("=" * 80)

## Parse Benchmark File

Extract all benchmark results and custom metrics from the output file.

In [ ]:
def parse_benchmark_file(filepath):
    """
    Parse Go benchmark output file and extract all metrics.
    
    Returns:
        DataFrame with columns: name, iterations, ns_op, custom_metrics, bytes_op, allocs_op
    """
    results = []
    
    # Regex pattern for benchmark lines
    # Format: BenchmarkName-N   iterations   ns/op   [custom metrics]   B/op   allocs/op
    pattern = r'^(Benchmark\S+)\s+(\d+)\s+(\d+\.?\d*)\s+ns/op(.*)$'
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            match = re.match(pattern, line)
            
            if match:
                name = match.group(1)
                iterations = int(match.group(2))
                ns_op = float(match.group(3))
                rest = match.group(4).strip()
                
                # Parse custom metrics and memory stats
                custom_metrics = {}
                bytes_op = 0
                allocs_op = 0
                
                # Extract all key-value pairs (e.g., "17.76 MB/s", "144 B/op")
                metric_pattern = r'(\d+\.?\d*)\s+([A-Za-z_/]+)'
                for match in re.finditer(metric_pattern, rest):
                    value = float(match.group(1))
                    unit = match.group(2)
                    
                    if unit == 'B/op':
                        bytes_op = value
                    elif unit == 'allocs/op':
                        allocs_op = value
                    else:
                        # Store custom metrics (MB/s, cache_lines, etc.)
                        custom_metrics[unit] = value
                
                results.append({
                    'name': name,
                    'iterations': iterations,
                    'ns_op': ns_op,
                    'bytes_op': bytes_op,
                    'allocs_op': allocs_op,
                    **custom_metrics
                })
    
    df = pd.DataFrame(results)
    
    # Add computed columns
    if not df.empty:
        # Extract benchmark category and variant
        df['category'] = df['name'].str.extract(r'Benchmark([^/]+)')[0]
        df['variant'] = df['name'].str.extract(r'/([^-]+)-\d+$')[0]
        
        # Extract size if present
        df['size'] = df['name'].str.extract(r'Size_(\d+)')[0]
        df['size'] = pd.to_numeric(df['size'], errors='coerce')
        
        # Compute throughput (ops/sec)
        df['ops_per_sec'] = 1e9 / df['ns_op']
    
    return df

# Parse the benchmark file
df = parse_benchmark_file(BENCHMARK_FILE)

print(f"\nParsed {len(df)} benchmark results")
print(f"\nColumns: {', '.join(df.columns)}")
print(f"\nBenchmark categories: {', '.join(df['category'].unique())}")

# Display first few rows
df.head(10)

## Summary Statistics

In [ ]:
# Overall statistics
print("=" * 80)
print("BENCHMARK SUMMARY")
print("=" * 80)

print(f"\nTotal benchmarks: {len(df)}")
print(f"Categories: {len(df['category'].unique())}")
print(f"\nPerformance range:")
print(f"  Fastest: {df['ns_op'].min():.2f} ns/op ({df.loc[df['ns_op'].idxmin(), 'name']})")
print(f"  Slowest: {df['ns_op'].max():.2f} ns/op ({df.loc[df['ns_op'].idxmax(), 'name']})")

if 'MB/s' in df.columns:
    print(f"\nThroughput range:")
    print(f"  Highest: {df['MB/s'].max():.2f} MB/s")
    print(f"  Lowest: {df['MB/s'].min():.2f} MB/s")

print(f"\nMemory allocation:")
print(f"  Max bytes/op: {df['bytes_op'].max():.0f} B/op")
print(f"  Max allocs/op: {df['allocs_op'].max():.0f} allocs/op")
print(f"  Zero-allocation benchmarks: {len(df[df['allocs_op'] == 0])} / {len(df)}")

## SIMD vs Fallback Comparison

Analyze performance differences between SIMD and fallback implementations.

In [ ]:
# Filter SIMD comparison benchmarks
simd_df = df[df['name'].str.contains('SIMDvsScalar', na=False)].copy()

if not simd_df.empty:
    # Extract operation type and implementation
    simd_df['operation'] = simd_df['name'].str.extract(r'/(\w+)_Size')[0]
    simd_df['implementation'] = simd_df['name'].str.extract(r'/(SIMD|Fallback)-')[0]
    
    # Pivot for comparison
    comparison = simd_df.pivot_table(
        index=['size', 'operation'],
        columns='implementation',
        values='ns_op'
    ).reset_index()
    
    if 'SIMD' in comparison.columns and 'Fallback' in comparison.columns:
        comparison['speedup'] = comparison['Fallback'] / comparison['SIMD']
        comparison['improvement_pct'] = (comparison['speedup'] - 1) * 100
        
        print("\n" + "=" * 80)
        print("SIMD PERFORMANCE ANALYSIS")
        print("=" * 80)
        print(f"\nAverage SIMD speedup: {comparison['speedup'].mean():.2f}x")
        print(f"Best SIMD speedup: {comparison['speedup'].max():.2f}x (size {comparison.loc[comparison['speedup'].idxmax(), 'size']})")
        print(f"Worst SIMD speedup: {comparison['speedup'].min():.2f}x (size {comparison.loc[comparison['speedup'].idxmin(), 'size']})")
        
        print("\nSpeedup by operation:")
        op_speedup = comparison.groupby('operation')['speedup'].mean().sort_values(ascending=False)
        for op, speedup in op_speedup.items():
            print(f"  {op:15s}: {speedup:.2f}x")
        
        display(comparison.sort_values('speedup', ascending=False))
else:
    print("No SIMD comparison benchmarks found in this file.")

## Visualization: SIMD Speedup by Size

In [ ]:
if not simd_df.empty and 'speedup' in comparison.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Speedup by size for each operation
    for operation in comparison['operation'].unique():
        op_data = comparison[comparison['operation'] == operation]
        ax1.plot(op_data['size'], op_data['speedup'], marker='o', label=operation, linewidth=2)
    
    ax1.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='No speedup')
    ax1.set_xscale('log')
    ax1.set_xlabel('Data Size (bytes)', fontsize=12)
    ax1.set_ylabel('Speedup (Fallback / SIMD)', fontsize=12)
    ax1.set_title('SIMD Speedup vs Data Size', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Speedup by operation (bar chart)
    op_speedup = comparison.groupby('operation')['speedup'].mean().sort_values(ascending=True)
    colors = ['green' if x > 1 else 'red' for x in op_speedup.values]
    op_speedup.plot(kind='barh', ax=ax2, color=colors, alpha=0.7)
    ax2.axvline(x=1, color='red', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Average Speedup (x)', fontsize=12)
    ax2.set_title('Average SIMD Speedup by Operation', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    
    # Save the figure
    graph_path = os.path.join(output_dir, 'graphs', 'simd_speedup_analysis.png')
    plt.savefig(graph_path, dpi=300, bbox_inches='tight')
    print(f"✓ Graph saved: {graph_path}")
    
    # Also save as SVG for high-quality reports
    svg_path = os.path.join(output_dir, 'graphs', 'simd_speedup_analysis.svg')
    plt.savefig(svg_path, format='svg', bbox_inches='tight')
    print(f"✓ Graph saved (SVG): {svg_path}")
    
    plt.show()
else:
    print("Cannot plot SIMD comparison - no data available")

## Visualization: Performance Scaling

In [ ]:
# Filter benchmarks with size information
sized_df = df[df['size'].notna()].copy()

if not sized_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: ns/op vs size
    for category in sized_df['category'].unique()[:5]:  # Limit to 5 categories
        cat_data = sized_df[sized_df['category'] == category]
        ax1.plot(cat_data['size'], cat_data['ns_op'], marker='o', label=category, linewidth=2)
    
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlabel('Size', fontsize=12)
    ax1.set_ylabel('Time (ns/op)', fontsize=12)
    ax1.set_title('Performance Scaling', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Throughput
    for category in sized_df['category'].unique()[:5]:
        cat_data = sized_df[sized_df['category'] == category]
        ax2.plot(cat_data['size'], cat_data['ops_per_sec'], marker='o', label=category, linewidth=2)
    
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.set_xlabel('Size', fontsize=12)
    ax2.set_ylabel('Throughput (ops/sec)', fontsize=12)
    ax2.set_title('Throughput Scaling', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save the figure
    graph_path = os.path.join(output_dir, 'graphs', 'performance_scaling.png')
    plt.savefig(graph_path, dpi=300, bbox_inches='tight')
    print(f"✓ Graph saved: {graph_path}")
    
    svg_path = os.path.join(output_dir, 'graphs', 'performance_scaling.svg')
    plt.savefig(svg_path, format='svg', bbox_inches='tight')
    print(f"✓ Graph saved (SVG): {svg_path}")
    
    plt.show()
else:
    print("No size-based benchmarks found for scaling analysis")

## Visualization: Memory Allocation Analysis

In [ ]:
# Filter benchmarks with memory allocations
mem_df = df[df['bytes_op'] > 0].copy()

if not mem_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Memory allocation by category (top 10)
    mem_by_cat = mem_df.groupby('category')['bytes_op'].mean().sort_values(ascending=False).head(10)
    mem_by_cat.plot(kind='barh', ax=ax1, color='steelblue', alpha=0.7)
    ax1.set_xlabel('Average Bytes/op', fontsize=12)
    ax1.set_title('Memory Allocation by Category', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Plot 2: Allocations vs Performance
    ax2.scatter(mem_df['allocs_op'], mem_df['ns_op'], alpha=0.6, s=100)
    ax2.set_xlabel('Allocations/op', fontsize=12)
    ax2.set_ylabel('Time (ns/op)', fontsize=12)
    ax2.set_title('Allocations vs Performance', fontsize=14, fontweight='bold')
    ax2.set_yscale('log')
    if mem_df['allocs_op'].max() > 10:
        ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save the figure
    graph_path = os.path.join(output_dir, 'graphs', 'memory_allocation_analysis.png')
    plt.savefig(graph_path, dpi=300, bbox_inches='tight')
    print(f"✓ Graph saved: {graph_path}")
    
    svg_path = os.path.join(output_dir, 'graphs', 'memory_allocation_analysis.svg')
    plt.savefig(svg_path, format='svg', bbox_inches='tight')
    print(f"✓ Graph saved (SVG): {svg_path}")
    
    plt.show()
    
    # Zero-allocation performance
    zero_alloc = df[df['allocs_op'] == 0]
    print(f"\nZero-allocation benchmarks: {len(zero_alloc)} / {len(df)}")
    print(f"Average performance (zero-alloc): {zero_alloc['ns_op'].mean():.2f} ns/op")
    print(f"Average performance (with-alloc): {mem_df['ns_op'].mean():.2f} ns/op")
else:
    print("No memory allocation data found")

## Hybrid Mode Analysis

Analyze array vs map mode performance if available.

In [ ]:
hybrid_df = df[df['name'].str.contains('HybridModes', na=False)].copy()

if not hybrid_df.empty:
    # Extract mode (Array/Map) and operation
    hybrid_df['mode'] = hybrid_df['name'].str.extract(r'(Array|Map)')[0]
    hybrid_df['operation'] = hybrid_df['name'].str.extract(r'(Add|Contains)')[0]
    
    # Extract size information from name (e.g., "Small_1K", "Large_1M")
    size_map = {'1K': 1000, '10K': 10000, '100K': 100000, '1M': 1000000, '10M': 10000000}
    hybrid_df['size_label'] = hybrid_df['name'].str.extract(r'(Small|Medium|Large)_(\d+[KM])')[1]
    hybrid_df['size_numeric'] = hybrid_df['size_label'].map(size_map)
    
    print("\n" + "=" * 80)
    print("HYBRID MODE ANALYSIS")
    print("=" * 80)
    
    # Compare Array vs Map performance
    if 'MB/s' in hybrid_df.columns:
        print("\nThroughput comparison:")
        mode_perf = hybrid_df.groupby('mode')['MB/s'].agg(['mean', 'min', 'max'])
        print(mode_perf)
    
    print("\nLatency comparison:")
    mode_latency = hybrid_df.groupby('mode')['ns_op'].agg(['mean', 'min', 'max'])
    print(mode_latency)
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Performance by mode and operation
    # Only plot if we have numeric size data
    if hybrid_df['size_numeric'].notna().any():
        pivot = hybrid_df.pivot_table(index='size_numeric', columns=['mode', 'operation'], values='ns_op')
        if not pivot.empty and pivot.shape[1] > 0:
            pivot.plot(ax=ax1, marker='o', linewidth=2)
            ax1.set_xlabel('Size', fontsize=12)
            ax1.set_ylabel('Time (ns/op)', fontsize=12)
            ax1.set_title('Hybrid Mode Performance', fontsize=14, fontweight='bold')
            ax1.set_xscale('log')
            ax1.legend(loc='best')
            ax1.grid(True, alpha=0.3)
        else:
            ax1.text(0.5, 0.5, 'Insufficient data for pivot plot', 
                    ha='center', va='center', transform=ax1.transAxes)
    else:
        # Alternative: bar plot by benchmark name
        plot_df = hybrid_df.sort_values('ns_op')
        x_pos = np.arange(len(plot_df))
        colors = ['skyblue' if 'Array' in name else 'coral' for name in plot_df['name']]
        ax1.barh(x_pos, plot_df['ns_op'], color=colors, alpha=0.7)
        ax1.set_yticks(x_pos)
        ax1.set_yticklabels(plot_df['name'].str.replace('BenchmarkHybridModes/', ''), fontsize=8)
        ax1.set_xlabel('Time (ns/op)', fontsize=12)
        ax1.set_title('Hybrid Mode Performance', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='x')
    
    # Plot 2: Memory allocations by mode
    mode_mem = hybrid_df.groupby('mode')['bytes_op'].mean()
    if not mode_mem.empty and mode_mem.sum() > 0:
        mode_mem.plot(kind='bar', ax=ax2, color=['skyblue', 'coral'], alpha=0.7)
        ax2.set_ylabel('Average Bytes/op', fontsize=12)
        ax2.set_title('Memory Usage by Mode', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Mode', fontsize=12)
        ax2.tick_params(axis='x', rotation=0)
        ax2.grid(True, alpha=0.3, axis='y')
    else:
        # If no memory allocations, show performance comparison instead
        mode_perf = hybrid_df.groupby('mode')['ns_op'].mean()
        mode_perf.plot(kind='bar', ax=ax2, color=['skyblue', 'coral'], alpha=0.7)
        ax2.set_ylabel('Average Time (ns/op)', fontsize=12)
        ax2.set_title('Average Performance by Mode', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Mode', fontsize=12)
        ax2.tick_params(axis='x', rotation=0)
        ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    # Save the figure
    graph_path = os.path.join(output_dir, 'graphs', 'hybrid_mode_comparison.png')
    plt.savefig(graph_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Graph saved: {graph_path}")
    
    svg_path = os.path.join(output_dir, 'graphs', 'hybrid_mode_comparison.svg')
    plt.savefig(svg_path, format='svg', bbox_inches='tight')
    print(f"✓ Graph saved (SVG): {svg_path}")
    
    plt.show()
else:
    print("No hybrid mode benchmarks found")

## Top Performers and Bottlenecks

In [ ]:
print("=" * 80)
print("TOP 10 FASTEST BENCHMARKS")
print("=" * 80)
display(df.nsmallest(10, 'ns_op')[['name', 'ns_op', 'ops_per_sec', 'bytes_op', 'allocs_op']])

print("\n" + "=" * 80)
print("TOP 10 SLOWEST BENCHMARKS")
print("=" * 80)
display(df.nlargest(10, 'ns_op')[['name', 'ns_op', 'ops_per_sec', 'bytes_op', 'allocs_op']])

if 'MB/s' in df.columns:
    print("\n" + "=" * 80)
    print("HIGHEST THROUGHPUT BENCHMARKS")
    print("=" * 80)
    high_throughput = df[df['MB/s'].notna()].nlargest(10, 'MB/s')
    display(high_throughput[['name', 'MB/s', 'ns_op', 'bytes_op', 'allocs_op']])

## Export Results & Profile Analysis

Export all benchmark data, CPU profiles, and generate flamegraphs for the complete investigation.

In [ ]:
# Export all data to CSV files
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("=" * 80)
print("EXPORTING RESULTS")
print("=" * 80)

# Export CSV files
csv_file = os.path.join(output_dir, 'data', 'all_benchmarks.csv')
df.to_csv(csv_file, index=False)
print(f"\n✓ All benchmarks exported to: {csv_file}")

# Export SIMD comparison if available
if not simd_df.empty and 'speedup' in comparison.columns:
    simd_csv = os.path.join(output_dir, 'data', 'simd_comparison.csv')
    comparison.to_csv(simd_csv, index=False)
    print(f"✓ SIMD comparison exported to: {simd_csv}")
    
    # Export summary statistics
    summary_data = {
        'metric': ['Average Speedup', 'Best Speedup', 'Worst Speedup', 'Median Speedup'],
        'value': [
            f"{comparison['speedup'].mean():.2f}x",
            f"{comparison['speedup'].max():.2f}x",
            f"{comparison['speedup'].min():.2f}x",
            f"{comparison['speedup'].median():.2f}x"
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    summary_csv = os.path.join(output_dir, 'data', 'simd_summary.csv')
    summary_df.to_csv(summary_csv, index=False)
    print(f"✓ SIMD summary statistics exported to: {summary_csv}")

# Export hybrid mode analysis if available
if not hybrid_df.empty:
    hybrid_csv = os.path.join(output_dir, 'data', 'hybrid_modes.csv')
    hybrid_df.to_csv(hybrid_csv, index=False)
    print(f"✓ Hybrid mode data exported to: {hybrid_csv}")
    
    # Mode comparison summary
    mode_comparison = pd.DataFrame({
        'mode': ['Array', 'Map'],
        'avg_ns_op': [
            hybrid_df[hybrid_df['mode'] == 'Array']['ns_op'].mean(),
            hybrid_df[hybrid_df['mode'] == 'Map']['ns_op'].mean()
        ],
        'avg_bytes_op': [
            hybrid_df[hybrid_df['mode'] == 'Array']['bytes_op'].mean(),
            hybrid_df[hybrid_df['mode'] == 'Map']['bytes_op'].mean()
        ]
    })
    mode_csv = os.path.join(output_dir, 'data', 'mode_comparison.csv')
    mode_comparison.to_csv(mode_csv, index=False)
    print(f"✓ Mode comparison summary exported to: {mode_csv}")

# Export top performers
top_10_fastest = df.nsmallest(10, 'ns_op')[['name', 'ns_op', 'ops_per_sec', 'bytes_op', 'allocs_op']]
top_csv = os.path.join(output_dir, 'data', 'top_10_fastest.csv')
top_10_fastest.to_csv(top_csv, index=False)
print(f"✓ Top 10 fastest benchmarks exported to: {top_csv}")

# Export top 10 slowest
top_10_slowest = df.nlargest(10, 'ns_op')[['name', 'ns_op', 'ops_per_sec', 'bytes_op', 'allocs_op']]
slow_csv = os.path.join(output_dir, 'data', 'top_10_slowest.csv')
top_10_slowest.to_csv(slow_csv, index=False)
print(f"✓ Top 10 slowest benchmarks exported to: {slow_csv}")

# Export metadata
metadata = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'source_file': BENCHMARK_FILE,
    'total_benchmarks': len(df),
    'categories': len(df['category'].unique()),
    'fastest_ns_op': df['ns_op'].min(),
    'slowest_ns_op': df['ns_op'].max(),
    'zero_alloc_count': len(df[df['allocs_op'] == 0]),
    'with_alloc_count': len(df[df['allocs_op'] > 0])
}
metadata_df = pd.DataFrame([metadata])
meta_csv = os.path.join(output_dir, 'data', 'metadata.csv')
metadata_df.to_csv(meta_csv, index=False)
print(f"✓ Metadata exported to: {meta_csv}")

print(f"\n✓ All data files saved to: {os.path.join(output_dir, 'data')}")
print(f"✓ All graphs have been saved to: {os.path.join(output_dir, 'graphs')}")

# Initialize profile variables
prof_df = None
prof_metadata = None
func_df = None
calls_df = None
profile_metadata = None

## Generate Comprehensive HTML Report

Create a standalone HTML report with all visualizations and data tables embedded.

In [ ]:
import subprocess
import re

def analyze_pprof_file(prof_file):
    """
    Analyze a Go pprof CPU profile file.
    
    Returns:
        DataFrame with profile data
    """
    try:
        # Run go tool pprof to get top functions
        result = subprocess.run(
            ['go', 'tool', 'pprof', '-top', '-nodecount=50', prof_file],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode != 0:
            print(f"Error running pprof: {result.stderr}")
            return None, None
        
        output = result.stdout
        
        # Extract metadata
        metadata = {}
        for line in output.split('\n')[:10]:
            if 'Duration:' in line:
                metadata['duration'] = line.split('Duration:')[1].split(',')[0].strip()
                metadata['total_samples'] = line.split('Total samples =')[1].split('(')[0].strip()
            elif 'Type:' in line:
                metadata['type'] = line.split('Type:')[1].strip()
            elif 'Time:' in line:
                metadata['time'] = line.split('Time:')[1].strip()
        
        # Parse the profile data
        profile_data = []
        in_data_section = False
        
        for line in output.split('\n'):
            # Start parsing after the header
            if 'flat  flat%   sum%' in line:
                in_data_section = True
                continue
            
            if not in_data_section:
                continue
            
            # Parse data lines
            # Format: flat  flat%   sum%        cum   cum%  function_name
            match = re.match(r'\s*(\d+\.?\d*[a-z]*)\s+(\d+\.?\d+)%\s+(\d+\.?\d+)%\s+(\d+\.?\d*[a-z]*)\s+(\d+\.?\d+)%\s+(.+)', line)
            if match:
                profile_data.append({
                    'flat': match.group(1),
                    'flat_pct': float(match.group(2)),
                    'sum_pct': float(match.group(3)),
                    'cum': match.group(4),
                    'cum_pct': float(match.group(5)),
                    'function': match.group(6).strip()
                })
        
        if not profile_data:
            print("No profile data found")
            return None, None
        
        df = pd.DataFrame(profile_data)
        
        # Simplify function names for display
        df['function_short'] = df['function'].apply(lambda x: x.split('/')[-1] if '/' in x else x)
        df['function_short'] = df['function_short'].apply(lambda x: x[:60] + '...' if len(x) > 60 else x)
        
        return df, metadata
        
    except subprocess.TimeoutExpired:
        print("pprof analysis timed out")
        return None, None
    except FileNotFoundError:
        print("Error: 'go' command not found. Make sure Go is installed and in PATH.")
        return None, None
    except Exception as e:
        print(f"Error analyzing pprof file: {e}")
        return None, None


# =============================================================================
# CPU PROFILE ANALYSIS (.prof files)
# =============================================================================
if PROFILE_FILE and os.path.exists(PROFILE_FILE):
    print("\n" + "=" * 80)
    print(f"CPU PROFILE ANALYSIS: {os.path.basename(PROFILE_FILE)}")
    print("=" * 80)
    
    prof_df, prof_metadata = analyze_pprof_file(PROFILE_FILE)
    
    if prof_df is not None:
        print(f"\nProfile Metadata:")
        for key, value in prof_metadata.items():
            print(f"  {key}: {value}")
        
        print(f"\n✓ Parsed {len(prof_df)} functions from profile")
        print(f"\nTop 10 CPU Consumers:")
        display(prof_df.head(10)[['function_short', 'flat_pct', 'cum_pct']])
        
        # Visualize top functions
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Plot 1: Top 15 functions by flat %
        top_flat = prof_df.head(15)
        ax1.barh(range(len(top_flat)), top_flat['flat_pct'], color='steelblue', alpha=0.7)
        ax1.set_yticks(range(len(top_flat)))
        ax1.set_yticklabels(top_flat['function_short'], fontsize=9)
        ax1.set_xlabel('Flat % (self time)', fontsize=12)
        ax1.set_title('Top 15 Functions by Self Time', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='x')
        ax1.invert_yaxis()
        
        # Plot 2: Top 15 functions by cumulative %
        top_cum = prof_df.nlargest(15, 'cum_pct')
        ax2.barh(range(len(top_cum)), top_cum['cum_pct'], color='coral', alpha=0.7)
        ax2.set_yticks(range(len(top_cum)))
        ax2.set_yticklabels(top_cum['function_short'], fontsize=9)
        ax2.set_xlabel('Cumulative % (including callees)', fontsize=12)
        ax2.set_title('Top 15 Functions by Cumulative Time', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
        ax2.invert_yaxis()
        
        plt.tight_layout()
        
        # Save the figure
        graph_path = os.path.join(output_dir, 'graphs', 'cpu_profile_analysis.png')
        plt.savefig(graph_path, dpi=300, bbox_inches='tight')
        print(f"\n✓ Graph saved: {graph_path}")
        
        svg_path = os.path.join(output_dir, 'graphs', 'cpu_profile_analysis.svg')
        plt.savefig(svg_path, format='svg', bbox_inches='tight')
        print(f"✓ Graph saved (SVG): {svg_path}")
        
        plt.show()
        
        # Export profile data
        prof_csv = os.path.join(output_dir, 'data', 'cpu_profile.csv')
        prof_df.to_csv(prof_csv, index=False)
        print(f"✓ CPU profile data exported to: {prof_csv}")

else:
    print("\n" + "=" * 80)
    print("CPU PROFILE ANALYSIS SKIPPED")
    print("=" * 80)
    if PROFILE_FILE is None:
        print("\nNo CPU profile file specified for this investigation.")
    print("\nTo generate CPU profiles, run:")
    print("  go test -bench=BenchmarkName -cpuprofile=results/cpu.prof")


print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print("\nFor interactive profile visualization, use:")
print("  go tool pprof -http=:8080 results/cpu.prof")